In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

from datetime import datetime, timedelta
import math
import time

import warnings
warnings.filterwarnings("ignore")

In [2]:
start_date = datetime.now() - timedelta(1)
year = str(start_date.year)
month = str(start_date.month)
day = "0" + str(start_date.day) if start_date.day < 10 else str(start_date.day)

In [3]:
required_entries = 1250

In [4]:
option = webdriver.ChromeOptions()
option.add_argument("start-maximized")

In [5]:
links_lst = []
i = 0
while i < required_entries:
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=option)
    time.sleep(5)
    driver.get(f"https://www.euronews.com/{year}/{month}/{day}")
    len_articles = driver.find_element(By.XPATH, '//p[@class="c-block-listing__results"]/strong[1]').text
    pages = int(len_articles) / 30
    pages = math.ceil(pages)
    for k in range(pages):
        driver.get(f"https://www.euronews.com/{year}/{month}/{day}?p={k+1}")
        articles_webscrapped = driver.find_elements(By.CLASS_NAME, "m-object__title__link ")
        for article in articles_webscrapped :
            links_lst.append(article.get_attribute('href'))
    driver.quit()
    i += int(len_articles)
    print(f"Added {i} links to the list")
    start_date -= timedelta(1)
    year = str(start_date.year)
    month = str(start_date.month)
    day = "0" + str(start_date.day) if start_date.day < 10 else str(start_date.day)
print(links_lst)

Added 60 links to the list
Added 115 links to the list
Added 173 links to the list
Added 232 links to the list
Added 295 links to the list
Added 312 links to the list
Added 340 links to the list
Added 390 links to the list
Added 463 links to the list
Added 542 links to the list
Added 606 links to the list
Added 661 links to the list
Added 685 links to the list
Added 705 links to the list
Added 758 links to the list
Added 817 links to the list
Added 881 links to the list
Added 940 links to the list
Added 992 links to the list
Added 1007 links to the list
Added 1028 links to the list
Added 1084 links to the list
Added 1141 links to the list
Added 1197 links to the list
Added 1254 links to the list
['https://www.euronews.com/video/2024/05/03/highway-collapse-in-southern-china-kills-at-least-36-people', 'https://www.euronews.com/2024/05/03/russian-trainers-relocate-to-niger-airbase-where-some-us-troops-still-stationed', 'https://www.euronews.com/my-europe/2024/05/03/situation-dire-for-ukra

In [6]:
from bs4 import BeautifulSoup

In [10]:
errors = []
list_of_articles = []
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=option)
time.sleep(5)
for article in links_lst:
    try :  
        driver.get(article)
        soup = BeautifulSoup(driver.page_source,'html.parser')
        title =  driver.find_elements(By.CLASS_NAME, "c-article-redesign-title")[0].text
        if not title:
            title =  driver.find_elements(By.CLASS_NAME, "c-article-redesign-title")[1].text

        contributors_div = soup.find('div', class_='c-article-contributors')
        bold_tags = contributors_div.find_all('b')
        authors = [tag.text.strip() for tag in bold_tags]
        
        if not authors:
            contributors_div = soup.find('div', class_='c-article-contributors')
            author_link = contributors_div.find('a')
            authors = author_link.text.strip()
               
        date = driver.find_elements(By.CLASS_NAME, "c-article-publication-date")[0].get_attribute('datetime') 
        
        divs = soup.find_all('div', class_='u-flex-container u-flex--wrap u-padding-bottom-2')
        try:
            second_div = divs[1]
        except:
            second_div = divs[0]
            
        a_tags = second_div.find_all('a')
        tags = [tag.get_text(strip=True) for tag in a_tags]

        dictionnary_of_article = {"title": title, "authors": authors, "date": date, "tags": tags, "link": article }
        list_of_articles.append(dictionnary_of_article)
        time.sleep(4)

    except Exception as e: 
        errors.append({article: e})

print(f"list_of_articles:\n{list_of_articles}\nerrors:{errors}")
driver.quit()

list_of_articles:
[{'title': 'Russian trainers relocate to Niger airbase where some US troops still stationed', 'authors': ['Euronews', 'AP'], 'date': '2024-05-04', 'tags': ['Niger', 'Russia', 'US troops', 'military junta', 'US politics', 'military forces'], 'link': 'https://www.euronews.com/2024/05/03/russian-trainers-relocate-to-niger-airbase-where-some-us-troops-still-stationed'}, {'title': "Situation 'dire' for Ukraine despite delayed US aid package", 'authors': 'Méabh Mc Mahon', 'date': '2024-05-04', 'tags': ['Ukraine', 'Ukraine war', 'Russia-Ukraine invasion'], 'link': 'https://www.euronews.com/my-europe/2024/05/03/situation-dire-for-ukraine-despite-delayed-us-aid-package'}, {'title': 'Judo Grand Slam: day one at Dushanbe', 'authors': ['Euronews', 'AP'], 'date': '2024-05-03', 'tags': ['Judo', 'Sport', 'Tajikistan'], 'link': 'https://www.euronews.com/2024/05/03/judo-grand-slam-day-one-at-dushanbe'}, {'title': "'Today's Ukraine could be tomorrow's East Asia': Japan and France agree

In [11]:
import pandas as pd

df = pd.DataFrame(list_of_articles, columns=["title", "authors", "date", "tags", "link"])

df.to_csv('articles_data.csv', index=False)

print("Dataset written")

Dataset written


In [12]:
df.head(10)

,title,authors,date,tags,link
0,Russian trainers relocate to Niger airbase whe...,"[Euronews, AP]",2024-05-04,"[Niger, Russia, US troops, military junta, US ...",https://www.euronews.com/2024/05/03/russian-tr...
1,Situation 'dire' for Ukraine despite delayed U...,Méabh Mc Mahon,2024-05-04,"[Ukraine, Ukraine war, Russia-Ukraine invasion]",https://www.euronews.com/my-europe/2024/05/03/...
2,Judo Grand Slam: day one at Dushanbe,"[Euronews, AP]",2024-05-03,"[Judo, Sport, Tajikistan]",https://www.euronews.com/2024/05/03/judo-grand...
3,'Today's Ukraine could be tomorrow's East Asia...,"[Euronews, IJF]",2024-05-03,"[Japanese politics, Fumio Kishida, South China...",https://www.euronews.com/2024/05/03/todays-ukr...
4,State of the Union: Enlargement nostalgia and ...,Shona Murray,2024-05-03,"[European Union, Eurovision song contest, EU e...",https://www.euronews.com/my-europe/2024/05/03/...
5,Sweden's Sámi pay the price in Europe's quest ...,Sophia Khatsenkova,2024-05-03,"[Raw material, Sweden, Mining]",https://www.euronews.com/2024/05/03/swedens-sa...
6,Strack-Zimmermann: Far-right would use electio...,Bryan Carter,2024-05-03,"[European politics, European Union, EU Parliam...",https://www.euronews.com/2024/05/03/strack-zim...
7,Kremlin brands France and UK's Ukraine comment...,Shona Murray,2024-05-03,"[Bombardment, Russia, Ukraine, Ukraine war]",https://www.euronews.com/2024/05/03/kremlin-br...
8,Tiny particles of plastic pollution could be f...,Ian Smith,2024-05-04,"[Health, nanoplastics, Microplastics, lung, br...",https://www.euronews.com/green/2024/05/03/tiny...
9,Thousands of London cab drivers set to sue Ube...,[AP],2024-05-04,"[Taxi driver, UBER, Taxi]",https://www.euronews.com/business/2024/05/03/t...


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1134 entries, 0 to 1133
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    1134 non-null   object
 1   authors  1134 non-null   object
 2   date     1134 non-null   object
 3   tags     1134 non-null   object
 4   link     1134 non-null   object
dtypes: object(5)
memory usage: 44.4+ KB
